In [16]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import soundfile as sf
import torch
from jiwer import wer
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt
import librosa

In [2]:
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [42]:
ds = load_dataset("kresnik/zeroth_korean", "clean")

In [46]:
ds['test'][2]

{'file': 'C:/Users/User/.cache/huggingface/datasets/downloads/extracted/ced79e66418d9fe8058679e42f8eac8957dd37121e912335283d1e45da227fdf/test_data_01/003/104\\104_003_0253.flac',
 'audio': {'path': 'C:/Users/User/.cache/huggingface/datasets/downloads/extracted/ced79e66418d9fe8058679e42f8eac8957dd37121e912335283d1e45da227fdf/test_data_01/003/104\\104_003_0253.flac',
  'array': array([-0.00030518, -0.00082397, -0.00100708, ..., -0.00012207,
          0.00018311, -0.00012207]),
  'sampling_rate': 16000},
 'text': '현재 백화점과 영화관 등은 오픈해 영업하고 있고 테마파크 및 아파트 등의 공사는 이천 십 팔 년 완공을 목표로 진행돼 왔다',
 'speaker_id': 104,
 'chapter_id': 3,
 'id': '104_003_0253'}

In [48]:
test_ds = ds['test']

def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

test_ds = test_ds.map(map_to_array)

def map_to_pred(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to("cuda")
    
    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

result = test_ds.map(map_to_pred, batched=True, batch_size=16, remove_columns=["speech"])
    
print("WER:", wer(result["text"], result["transcription"]))

Map: 100%|██████████| 457/457 [01:54<00:00,  4.01 examples/s]

WER: 0.04728203583797621


In [52]:
print(result["text"][1],result["transcription"][1])

그 바람에 나의 몸도 겹쳐 쓰러지며 한창 피어난 노란 동백꽃 속으로 폭 파묻혀 버렸다 그 바람에 나의 몸두 겹쳐로 쓰러지며 한창 피어난 노란 동백꽃 속으로 폭 파묻혀 버렸다


In [54]:
sum(list(map(int,[result["text"][i]!=result["transcription"][i] for i in range(len(result["text"]))])))

146

In [47]:

def get_pred(x):
        speech, _ = sf.read(x["file"])
        x["speech"] = speech
        inputs = processor(x["speech"],sampling_rate=16000,return_tensors="pt",padding="longest")
        input_values = inputs.input_values.to("cuda")
        with torch.no_grad():
                logits = model(input_values).logits
        pred_id = torch.argmax(logits,dim=-1)
        transcription = processor.batch_decode(pred_id)
        x["transcription"] = transcription
        return x["transcription"]


FILE_PATH = './FVA/target_sounds/pungdung.wav'
FILE_PATH = ds['test'][2]['file']
x = librosa.load(path=FILE_PATH, sr=16000)[0]
input_data = {
        'file':FILE_PATH,
        'audio':{'path':FILE_PATH,
                'array':np.array(x),
                'sampling_rate':16000},
        'speaker_id':0,
        'chapter_id':0,
        'id':0
        }

print(get_pred(input_data))


['현재 백화점과 영화관 등은 오픈해 영업하고 있고 테마파크 및 아파트 등의 공사는 이천 십 팔 년 완공을 목표로 진행돼 왔다']
